In [ ]:
import xarray as xr
import pickle

In [ ]:
ds = xr.open_dataset('Data\92l.2023082400.hfsa.storm.atm.f024.grb2')

In [ ]:
drop_variables = ['q', 'w', 'wz', 'absv', 'clwmr', 'icmr', 'rwmr', 'snmr','grle', 'rare']
ds_atm = xr.open_dataset('Data\92l.2023082400.hfsa.storm.atm.f024.grb2',
                         drop_variables = drop_variables, 
                         filter_by_keys={'typeOfLevel': 'isobaricInhPa'})

ds_sfc = xr.open_dataset('Data\92l.2023082400.hfsa.storm.atm.f024.grb2',
                         drop_variables = drop_variables,  
                         filter_by_keys={'typeOfLevel': 'meanSea'})

In [ ]:
cmp = ds_sfc['prmsl'].min().values

In [ ]:
ds_atm.sel

In [ ]:
ds_atm.sel(isobaricInhPa = cmp, method = 'nearest')

In [ ]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config

def list_s3_files(bucket_name, prefix=''):
    """
    Lists files in an AWS S3 bucket, optionally filtered by a prefix.

    Args:
        bucket_name (str): The name of the S3 bucket.
        prefix (str, optional): An optional prefix to filter files (e.g., 'folder/').
    """
    s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
    
    # Handle pagination for more than 1000 objects
    paginator = s3_client.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket_name, Prefix=prefix)
    file_list = []
    for page in pages:
        if 'Contents' in page:
            for obj in page['Contents']:
                file_list.append(obj)
        else:
            print(f"No files found in '{bucket_name}' with prefix '{prefix}'.")
    return file_list

In [ ]:
list = list_s3_files('noaa-nws-hafs-pds', prefix = 'hfsa')

In [ ]:
with open('link_list.pkl', 'wb') as f:
    pickle.dump(list, f)

In [71]:
with open('Data/links/link_list.txt', 'w') as f:
    for item in list:
        if ('f024' in item['Key'] or 'f027' in item['Key']) and 'idx' not in item['Key']:
            f.write(item['Key'] + '\n')

In [ ]:
list